In [6]:
import numpy as np
import numpy as np
import cv2
import math
from scipy import signal
from skimage.filters import frangi, hessian

img = cv2.imread('source/0249478-18539.jpg',1)
width = int(img.shape[1]*0.3)
height = int(img.shape[0]*0.3)
img = cv2.resize(img, (width,height), interpolation = cv2.INTER_AREA)
b,g,r = cv2.split(img)

#preprocess ---enchnace and dilute blood vessel
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
g = clahe.apply(g)
kernel15 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(15,15))
img_dilation = cv2.dilate(g, kernel15, iterations=1) 
img_dilation_erosion = cv2.erode(img_dilation, kernel15, iterations=1) 

probe1 = img_dilation_erosion.copy()
#preprocess ----get black outtra circle's coordinate
out_cicleB_mask = np.array(g,dtype=np.uint8)
for i in range(height):
    for j in range(width):
        if g[i,j]<=50:
                out_cicleB_mask[i,j] = 1
            
#finding seed point
g_convert = 255 - g
max_intens = np.amax(g_convert)
for i in range(height):
    for j in range(width):
        if max_intens == g_convert[i,j]:
            seedX, seedY = i,j

#growing seed
sample= 255 - img_dilation_erosion
grow_seed = np.array(sample,dtype=np.uint8)
for i in range(height):
    for j in range(width):
        if np.abs(sample[seedX,seedY] - sample[i,j])>= 220:
            grow_seed[i,j]= 255
        else: grow_seed[i,j] = 0

#set outer circle as black, so as date
for i in range(20):
    for j in range(width):
        grow_seed[i,j] = 0
for i in range(height):
    for j in range(width):
        if out_cicleB_mask[i,j] ==1:
            grow_seed[i,j] = 0


#contour and drawing pad
kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
close=cv2.morphologyEx(grow_seed,cv2.MORPH_CLOSE,kernel)
contours, hierarchy = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

if len(contours) != 0:
    for c in contours:
        if len(c) >= 50:
            hull=cv2.convexHull(c)
            cv2.ellipse(g, cv2.fitEllipse(hull),(0,0,0),1)
            _, (MA, ma),_ = cv2.fitEllipse(hull)
            area_in = math.pi*MA*ma

############################################## Disc cal #####################################################
#threshold = img_dilation_erosion > 180
#threshold = threshold.astype(np.uint8)
#for i in range(height):
#    for j in range(width):
#        if threshold[i,j]==1:
#            threshold[i,j] = 255
#
#threshold = cv2.dilate(threshold, kernel15, iterations=1) 
#threshold = cv2.erode(threshold, kernel15, iterations=1)
            
thresholdR = r >= 220
thresholdR = thresholdR.astype(np.uint8)
for i in range(height):
    for j in range(width):
        if thresholdR[i,j]==1:
            thresholdR[i,j] = 255
thresholdR = cv2.medianBlur(thresholdR,15)

close01=cv2.morphologyEx(thresholdR,cv2.MORPH_CLOSE,kernel)
contours01, hierarchy01 = cv2.findContours(close01, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
if len(contours01) != 0:
    for c in contours01:
        if len(c) >= 50:
            hull=cv2.convexHull(c)
            cv2.ellipse(g, cv2.fitEllipse(hull),(0,0,0),1)
            _, (MA01, ma01),_ = cv2.fitEllipse(hull)
            area_out = math.pi*MA01*ma01

print('C:D = ',area_in/area_out)
cv2.imshow('img',g)
cv2.imshow('img after',thresholdR)
cv2.waitKey(0) 
cv2.destroyAllWindows()

C:D =  0.11893958529188507


In [7]:
cv2.imshow("img",probe1)
cv2.waitKey(0) 
cv2.destroyAllWindows()